# Scraping 101

# Kittens

In [1]:
import requests 

try:
    request = requests.get("http://placekitten.com/")
    kittens = request.text
    print (kittens[559:1000])
except requests.exceptions.RequestException as e:
    print ("No kittenz. Got an error code:", e)

                      __     __,
                      \,`~"~` /
      .-=-.           /    . .\
     / .-. \          {  =    Y}=
    (_/   \ \          \      / 
           \ \        _/`'`'`b
            \ `.__.-'`        \-._
             |            '.__ `'-;_
             |            _.' `'-.__)
              \    ;_..-`'/     //  \
              |   /  /   |     //    |
              \  \ \__)   \   //    /
               \__)  


# PDF

## Download pdf

Импортируем модуль requests и откроем файл для записи, в который потом сохраним нашу pdf

In [1]:
import requests

o = open("press.pdf",'wb')

Мы будем сохранять отчет Евро Парламента. У нас уже есть ссылка на саму pdf, поэтому просто укажем ее. С помощью модуля requests откроем ссылку для потокового скачивания файла (размер pdf может быть большим и не факт что он уместиться за раз в памяти)

In [2]:
url=("http://www.europarl.europa.eu/pdfs/news/public/story/20161026STO49002/20161026STO49002_en.pdf" )
release=requests.get(url, stream=True)
press_release=release.content

Осталось все записать файл и закрыть его запись

In [3]:
o.write(press_release)
o.close()

## Download all PDFs from page

Попробуем задание посложнее. Скачаем **все** pdf, которые есть на странице с курсами ВШЭ для направления политология. Зайдя на сайт и выбрав необходимые фильтры, получаем следующую страницы, на которой есть инормация о курсах:

https://www.hse.ru/edu/courses/index.html?words=&full_words=&edu_year=2018&lecturer=&edu_level=78397&language=&level=1191462%3A130671177&mandatory=&is_dpo=0&filial=22723&modules=10162246&modules=10162247&xlc=&genelective=0

In [4]:
import requests
import bs4

url = ("https://www.hse.ru/edu/courses/index.html?words=&full_words=&edu_year=2018&lecturer=&edu_level=78397&language=&level=1191462%3A130671177&mandatory=&is_dpo=0&filial=22723&modules=10162246&modules=10162247&xlc=&genelective=0")
page = requests.get(url)
courses = page.text

Используем модуль bs4, а конкретно метод BeautifulSoup, который представит нам сайт в удобном для работы виде (информация будет разбита по тэгам веб страницы div, a и т.д.)

В списке links будет хранить ссылки на наши pdf файлы с курсами.

In [5]:
links = [] 

soup = bs4.BeautifulSoup(courses, "html.parser")

Изучив код страницы, мы видим что ссылки на pdf лежат в теге "а". Поэтому найдем все такие теги и заберем адрес ссылки, которая хранится внутри тега "а" с параметром href. Проверим, ссылка ли это на курс (в этом теге могут быть ссылки на элементы меню) и есть ли такая ссылка в нашем списке, и если нет, то добавим ее.

**Всегда** придется изучать строение самой таблицы, чтобы обращаться и брать ее элементы.



In [8]:
for name in soup.findAll ("a"):
    course = name.get ("href")
    if "hse.ru/data/" in str(course):
        if str(course) not in links:
            links.append(course)

In [9]:
links

['https://www.hse.ru/data/2019/09/03/1491358191/program-2855895948-0wXzMk9PI5.pdf',
 'https://www.hse.ru/data/2019/10/07/1536829037/program-3545935072-ANrx0EiOCZ.pdf',
 'https://www.hse.ru/data/2019/09/09/1491361404/program-2844123818-oQud3m8EjA.pdf',
 'https://www.hse.ru/data/2019/09/09/1491361376/program-2874011060-IlSg71As3V.pdf',
 'https://www.hse.ru/data/2019/07/03/1491340575/program-2856872089-k8pvU_GvCU.pdf',
 'https://www.hse.ru/data/2019/09/12/1491598376/program-2829284138-RKNJlnkM1d.pdf',
 'https://www.hse.ru/data/2019/10/14/1491597605/program-2874731637-Tg4F0JMfUU.pdf',
 'https://www.hse.ru/data/2019/09/15/1491597601/program-2883828071-16cqW_EkxJ.pdf',
 'https://www.hse.ru/data/2019/07/03/1491341093/program-2880421577-z1Iwqcp4SM.pdf',
 'https://www.hse.ru/data/2019/09/05/1491341097/program-2874707073-REi4Mn2MR8.pdf',
 'https://www.hse.ru/data/2019/09/13/1491338598/program-2859717269-TKeLNuPKR3.pdf',
 'https://www.hse.ru/data/2019/11/10/1531702074/program-2853069879-sMJoRTH1z

Получили список ссылок на pdf со **всей** страницы.

Теперь мы пройдемся по всем этим ссылкам и точно также как при скачивании pdf с сайта Евро Парламента просто откроем новым requests'ом и запишем в файл. Название файла будет точно таким же как он назывался на сайте.

In [10]:
for link in links: 
    course = requests.get(link, stream=True)
    course_content = course.content
    o = open("" + link[link.rfind("/") + 1:], "wb") # link[link.rfind("/") + 1:] этим мы берем название файла из конца ссылки
    o.write(course_content)
o.close()

## Download all PDFs from several pages

Еще усложним задачу. Теперь мы будем собирать pdf с нескольких страницы выдачи. Код будет похож на прошлый пример и мы только обернем его циклом по страницам.

Заведем переменную *page* для хранения текущей страницы и будем увеличивать ее после завершения прохода текущей страницы.

Для выбора новой страницы просто подняем цифру в адресе сайт на нужную. Пример:
https://www.hse.ru/edu/courses/page2.html...
https://www.hse.ru/edu/courses/page3.html...
https://www.hse.ru/edu/courses/page4.html...

И т.д.


In [12]:
import requests
import bs4

page = 1
while page <= 3: #количество страниц, по которым нужно пройти
    
    #делаем запрос к нужной странице
    url = ("https://www.hse.ru/edu/courses/page" + str(page) + ".html?words=&full_words=&edu_year=2018&lecturer=&edu_level=78397&language=&level=1191462%3A130671177&mandatory=&is_dpo=0&filial=22723&modules=10162246&modules=10162247&xlc=&genelective=0")
    pages = requests.get(url)
    courses = pages.text
    
    #дальше код полностью с прошлого примера

    links = []  

    soup = bs4.BeautifulSoup(courses, "html.parser")

    for name in soup.findAll ("a"):
        course = name.get ("href")
        if "hse.ru/data/" in str(course):
            if str(course) not in links:
                links.append(course)
                
    for link in links: 
        course = requests.get(link, stream=True)
        course_content = course.content
        o = open("" + link[link.rfind("/") + 1:], "wb") # link[link.rfind("/") + 1:] этим мы берем название файла из конца ссылки
        o.write(course_content)
            
    page += 1 # "берем" новую страницу
    
o.close()

# Duma

В этом примере мы зайдем на сайт государственной думы и попробуем получить данные обо всех ее депутатах, в какой фракции и комитете он состоит, а также занимаю должность. Все это в итоге представим в виде DataFrame и сохраним как csv файл, для дальнейшей работы.

Делаем все как обычно, заходим на сайт думы, используем requests и BeatifulSoup

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

url = "http://duma.gov.ru/duma/deputies/7/"
page = requests.get(url)

text = page.text
soup = BeautifulSoup(text, "html.parser")

Имена депутатов будет хранить в deputates, комитеты в которых они состоят в commissions, должность в occupations и партию в party.

In [14]:
deputates = []
commissions = []
occupations = []
party = []

Депутатов много (487 человек) и скрейпинг может занимать прилично времени. Поэтому ограничим количество обработанных человек с помощью счетчика count и проверки. Для примера оставим 10 человек.

Данные обо всхе депутатах находятся в теге "a" и с классом "person__image-wrapper person__image-wrapper--s"

Найдем все такие теги

In [15]:
count = 0    
person = soup.find_all('a', class_ = "person__image-wrapper person__image-wrapper--s")

Теперь для каждого депутата мы должны получить его имя и ссылку, по которой мы пройдем и выясним остальную информацию: комитет, должность и фракцию.

К нашему счастью вместе с фото депутата в теге "img" хранится элемент  "alt" которой по наведению на фото, покажет в левом нижнем углу текст, который в нашем случае представляет собой Имя Фамилию и Отчество. Если бы не это, нам пришлось бы делать два поиска по тегам, т.к. имя хранится в одном теге, а фамилия и отчество в другом.

В том же теге "img" хранится ссылка на страницу с этим конкретным депутатом. Получим ее из элемента "href". Поэтому не нужно делать новый поиск тега.

Делаем обычный request к странице депутата, как мы делали в примере с PDF.

На странице с депутатом нам нужно найти тег в котором хранится его комитет и фракция. Посмотрев исходный код страницы, находим нужные теги - "a" с классами link "link--underline link--external" и "link link--underline person__description__link". Сохраняем их по своим массивам

Одна проблема, должность в комитете находится в строке с самим комитетом. Но к нашему счастью, они указываются в скобочках. Поэтому найдем индекс первого появления скобочек в строке с комитетом. Вырежем их и запишем в нужный массив. Точно также запишем комитет, обрезав у него конец строки

In [16]:
person[0]

<a class="person__image-wrapper person__image-wrapper--s" href="/duma/persons/99112936/" itemprop="url">
<picture>
<img alt="Авдеев Александр Александрович" class="person__image person__image--s" itemprop="image" src="/media/persons/240x240/OM9mmw6lAZ1lhnpM1yYBIMAo1BaQgxnz.jpg"/>
</picture>
</a>

In [17]:
for p in person:
    print(p.find('img'))
    name = p.find('img')
    print(name.get('alt'))
    print(p.get('href'))
    print(deputates)
    url_deput = "http://duma.gov.ru/" + p.get("href")
    print(url_deput)
    page = requests.get(url_deput)
    soup_deput = BeautifulSoup(page.text, "html.parser")
    comm = soup_deput.find('a', class_ = "link link--underline link--external")
    #print(comm)
    t = comm.text.strip()
    t_new = t[:t.find('(')]
    print(t_new)
    break

<img alt="Авдеев Александр Александрович" class="person__image person__image--s" itemprop="image" src="/media/persons/240x240/OM9mmw6lAZ1lhnpM1yYBIMAo1BaQgxnz.jpg"/>
Авдеев Александр Александрович
/duma/persons/99112936/
[]
http://duma.gov.ru//duma/persons/99112936/
Комитет по контролю и Регламенту 


In [18]:
t = comm.text.strip()
t[t.find('(')]

'('

In [19]:
for p in person:  
    
    # Обрезаем скрейпинг, т.к. очень долго идет обработка
    if count > 10:
        break
    
    #Получаем имена
    name = p.find('img', class_ = 'person__image person__image--s') # находим тег img и забираем из него значение alt
    deputates.append(name.get("alt"))
    
    # Получаем ссылку на депутата
    url_deput = "http://duma.gov.ru/" + p.get("href") # в нашем найденном теге уже есть нужная ссылка на депутата. Забираем ее
    
    # Делаем запрос
    page = requests.get(url_deput)
    soup_deput = BeautifulSoup(page.text, "html.parser")
    
    #Ищем комитет
    comm = soup_deput.find('a', class_ = "link link--underline link--external")
    t = comm.text.strip("\n")
    # регулярным выражением убираем все непечатный символы
    commissions.append(re.sub('\W+',' ',t[:t.index("(")-1]).strip()) # убираем должность
    occupations.append(t[t.index("(") + 1:t.index(")")]) #должность всегда в скобочках, 
                                                         #находим первые вхождени ( и )

    # Поиск фракции
    part = soup_deput.find('a', class_ = "link link--underline person__description__link")
    party.append(part.text)

    count += 1 # обновляем счетчик

Все данные получены, осталось создать DataFrame и сохранить его в виде csv

In [20]:
df = pd.DataFrame({'Депутат': deputates, 'Коммитет':commissions ,'Должность':occupations, 'Фракция': party})
df.to_csv("Gos_duma.csv")

Можно посмотреть на первые несколького элементов используя команду df.head()

In [21]:
df.head()

,Депутат,Коммитет,Должность,Фракция
0,Авдеев Александр Александрович,Комитет по контролю и Регламенту,член комитета,«ЕДИНАЯ РОССИЯ»
1,Авдеев Михаил Юрьевич,Комитет по транспорту и строительству,Первый заместитель председателя комитета,КПРФ
2,Агаев Ваха Абуевич,Комитет по финансовому рынку,заместитель председателя комитета,КПРФ
3,Адучиев Батор Канурович,Комитет по аграрным вопросам,член комитета,«ЕДИНАЯ РОССИЯ»
4,Азимов Рахим Азизбоевич,Комитет по безопасности и противодействию корр...,член комитета,«ЕДИНАЯ РОССИЯ»
